In [2]:
import logging
import warnings
import os
import pandas as pd
import numpy as np
# from vendi_score import vendi
import torch
import transformers
from datasets import Dataset,load_dataset
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    f1_score,
    matthews_corrcoef,
    precision_score,
    recall_score,
    roc_auc_score,
)
import json


transformers.logging.set_verbosity(transformers.logging.ERROR)
logging.disable(logging.ERROR)
warnings.filterwarnings("ignore", category=FutureWarning)

### Define metrics functions

In [4]:
# Define metrics
def compute_metrics_hf(preds):
    """computes F1 score and accuracy over dataset
    Args:
        model (any type): model for evaluation
        testing_dataloader (huggingface dataset): self explained
    Returns:
        dict
    """
    y_pred=preds.predictions
    y_true=preds.label_ids
    y_pred=y_pred.argmax(axis=1)
    mcc=matthews_corrcoef(y_true,y_pred)
    f1=f1_score(y_true,y_pred)
    precision=precision_score(y_true,y_pred)
    recall=recall_score(y_true,y_pred)
    roc_auc=roc_auc_score(y_true,y_pred)
    return{
        "mcc":mcc,
        "f1":f1,
        "precision":precision,
        "recall":recall,
        "roc_auc":roc_auc,
    }

def compute_metrics(testing_dataloader,model):
    """computes F1 score and accuracy over dataset
    Args:
        model (any type): model for evaluation
        testing_dataloader (huggingface dataset): self explained
    Returns:
        dict
    """
    y_true=[]
    y_pred=[]
    model.eval()
    
    for batch in testing_dataloader:
        batch={k:v.to(model.device)for k,v in batch.items()}

        batch["labels"] = batch["labels"].long()
        with torch.no_grad():
            outputs=model(**batch)
            
        logits=outputs.logits
        predictions=torch.argmax(logits,dim=-1)
        y_true.extend(batch["labels"].type(torch.LongTensor).tolist())  
        y_pred.extend(predictions.type(torch.LongTensor).tolist()) 
    mcc=matthews_corrcoef(y_true,y_pred)
    f1=f1_score(y_true,y_pred)
    precision=precision_score(y_true,y_pred)
    recall=recall_score(y_true,y_pred)
    roc_auc=roc_auc_score(y_true,y_pred)
    return{
        "mcc":mcc,
        "f1":f1,
        "precision":precision,
        "recall":recall,
        "roc_auc":roc_auc,
    }


### Define trainer function

In [5]:
# Define model - eventually use others? 

base_model = "roberta-base"
device = (
    torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
)

model = AutoModelForSequenceClassification.from_pretrained(
    base_model,
    num_labels=2,
)
tokenizer = AutoTokenizer.from_pretrained(base_model)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # dynammically pads sequences to the length of the longest sequence in each batch

# the pretrained head of roberta is discarded and replaced with a randomly initialized classification head

def tokenize_function(examples):
   return tokenizer(
        examples["text"], padding=True, truncation=True, max_length=128, return_tensors="pt"
    )

In [6]:
   
def train_subsamples(subsamples_dataset, dataset_name, test_set):
    results_list = []
    output_dir = f"./results/{dataset_name}"
    os.makedirs(output_dir, exist_ok=True)
    
    for name,df in subsamples_dataset.items():

        dev_df, test_df = train_test_split(test_set,test_size=0.67,random_state=42, stratify=test_set["label"])
        # Convert to HF datasets and tokenize
        train_df = Dataset.from_pandas(df).map(tokenize_function, batched=True)
        dev_df = Dataset.from_pandas(dev_df).map(tokenize_function, batched=True)
        test_df = Dataset.from_pandas(test_df).map(tokenize_function, batched=True)

        # Set the format for PyTorch
        train_df.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
        dev_df.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
        test_df.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

        def ensure_long_tensor(examples):
            examples["label"] = torch.tensor(examples["label"]).long()  # Convert labels to LongTensor
            return examples

        train_df = train_df.map(ensure_long_tensor, batched=True)
        dev_df = dev_df.map(ensure_long_tensor, batched=True)
        test_df = test_df.map(ensure_long_tensor, batched=True)
                
        training_args=TrainingArguments(
            output_dir="./checkpoints",
            per_device_eval_batch_size=32,
            per_device_train_batch_size=32,
            num_train_epochs=3,
            save_total_limit=3,
            evaluation_strategy="steps",
            logging_steps=50,
            eval_steps=50,
            save_steps=50,
            disable_tqdm=False,
            weight_decay=0.05,
            learning_rate=2e-5,
            run_name=f"{name}",
            metric_for_best_model="eval_loss",
            save_strategy="steps",
            load_best_model_at_end=True,
            remove_unused_columns=False,
        )
    
        # Trainer initialization
        trainer=Trainer(
            model,
            training_args,
            train_dataset=train_df,
            eval_dataset=dev_df,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics_hf,
            data_collator=data_collator,
            callbacks=[
                EarlyStoppingCallback(
                    early_stopping_patience=5,
                    early_stopping_threshold=0.0,
                ),
            ],
        )
    
        # Train the model
        trainer.train()
        
        # Save model for further analysis (like checklist?)
        # trainer.save_model(f"{output_dir}/{name}_model") 
        
        # Evaluate on the test set
        eval_dataloader=DataLoader(
            test_df,
            batch_size=32,
            collate_fn=data_collator,
            pin_memory=True
        )
    
        
        # Compute metrics
        metrics = compute_metrics(eval_dataloader, model)
        metrics['subsample'] = name 
        results_list.append(metrics)
    
        
    with open(f"{output_dir}/{dataset_name}_results.json", "w") as f:
        json.dump(results_list, f)


    return results_list


### Fine-tune for the subsamples (N=500) for the three datasets

In [7]:
# Load subsets 
annomatic_subsamples = {}
babe_subsamples = {}
basil_subsamples = {}

annomatic_folder = "/kaggle/input/annomatic-subsets"
babe_folder = "/kaggle/input/babe-subsets"
basil_folder = "/kaggle/input/basil-subsets"


def load_subsets(folder_path):
    subsets = {}
    for filename in os.listdir(folder_path):
        if filename.startswith("alpha"):  
            sample_name = filename.split(".")[0]  
            subsets[sample_name] = pd.read_csv(os.path.join(folder_path, filename))  
    return subsets

annomatic_subsamples = load_subsets(annomatic_folder)
babe_subsamples = load_subsets(babe_folder)
basil_subsamples = load_subsets(basil_folder)


# Test sets
babe_test = pd.read_csv("/kaggle/input/test-sets/babe_test.csv") 
basil_test = pd.read_csv("/kaggle/input/test-sets/basil_test.csv") 

In [49]:
results_annomatic = train_subsamples(
    subsamples_dataset=annomatic_subsamples,  
    dataset_name="annomatic",  
    test_set= babe_test
)

results_annomatic_df = pd.DataFrame(results_annomatic)
results_annomatic.to_csv("/kaggle/working/results_annomatic.csv")

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/tmp/ipykernel_30/1220641441.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  examples["label"] = torch.tensor(examples["label"]).long()  # Convert labels to LongTensor


...

/tmp/ipykernel_30/1220641441.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  examples["label"] = torch.tensor(examples["label"]).long()  # Convert labels to LongTensor


Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


In [46]:
results_babe = train_subsamples(
    subsamples_dataset=babe_subsamples,  
    dataset_name="babe",  
    test_set= babe_test
)

results_babe_df = pd.DataFrame(results_babe)
results_babe_df.to_csv("/kaggle/working/results_babe.csv")

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/tmp/ipykernel_30/1220641441.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  examples["label"] = torch.tensor(examples["label"]).long()  # Convert labels to LongTensor


...

/tmp/ipykernel_30/1220641441.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  examples["label"] = torch.tensor(examples["label"]).long()  # Convert labels to LongTensor


Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


In [35]:
results_basil = train_subsamples(
    subsamples_dataset=basil_subsamples,  
    dataset_name="basil",  
    test_set= basil_test
)

results_basil_df = pd.DataFrame(results_basil)
results_basil_df.to_csv("/kaggle/working/results_basil.csv")

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/tmp/ipykernel_30/1220641441.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  examples["label"] = torch.tensor(examples["label"]).long()  # Convert labels to LongTensor


...

/tmp/ipykernel_30/1220641441.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  examples["label"] = torch.tensor(examples["label"]).long()  # Convert labels to LongTensor


Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


#### Test with larger subsamples sizes

In [8]:
annomatic_1500_folder = "/kaggle/input/annomatic-1500-subsets"
annomatic_1500_subsamples = load_subsets(annomatic_1500_folder)

annomatic_1000_folder = "/kaggle/input/annomatic-1000-subsets"
annomatic_1000_subsamples = load_subsets(annomatic_1000_folder)

annomatic_200_folder = "/kaggle/input/annomatic-200-subsets"
annomatic_200_subsamples = load_subsets(annomatic_200_folder)

# Test sets
babe_test = pd.read_csv("/kaggle/input/test-sets/babe_test.csv") 


In [ ]:
results_annomatic_1500 = train_subsamples(
    subsamples_dataset=annomatic_1500_subsamples,  
    dataset_name="annomatic",  
    test_set= babe_test
)

results_annomatic_1500_df = pd.DataFrame(results_annomatic_1500)
results_annomatic_1500_df.to_csv("/kaggle/working/results_annomatic_1500.csv")

In [15]:
results_annomatic_1000 = train_subsamples(
    subsamples_dataset=annomatic_1000_subsamples,  
    dataset_name="annomatic",  
    test_set= babe_test
)

results_annomatic_1000_df = pd.DataFrame(results_annomatic_1000)
results_annomatic_1000_df.to_csv("/kaggle/working/results_annomatic_1000.csv")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/tmp/ipykernel_30/4080805664.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  examples["label"] = torch.tensor(examples["label"]).long()  # Convert labels to LongTensor


...

/tmp/ipykernel_30/4080805664.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  examples["label"] = torch.tensor(examples["label"]).long()  # Convert labels to LongTensor


Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


In [9]:
results_annomatic_200 = train_subsamples(
    subsamples_dataset=annomatic_200_subsamples,  
    dataset_name="annomatic",  
    test_set= babe_test
)

results_annomatic_200_df = pd.DataFrame(results_annomatic_200)
results_annomatic_200_df.to_csv("/kaggle/working/results_annomatic_200.csv")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/tmp/ipykernel_30/4080805664.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  examples["label"] = torch.tensor(examples["label"]).long()  # Convert labels to LongTensor


...

/tmp/ipykernel_30/4080805664.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  examples["label"] = torch.tensor(examples["label"]).long()  # Convert labels to LongTensor


Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
